# <center>Application of Ant Colony Optimization in FSP problem</center>

This notebook presents a practical approach to solving the flowshop problem by implementing the Ant Colony Optimization algorithm. These metaheuristics are effective in generating high-quality solutions for large instances of the problem, requiring only a reasonable amount of computational resources. Compared to heuristics, such metaheuristics are more effective for solving the flowshop problem because they can escape from local optima and find better solutions

# Table of Contents

1. [Johnson **n** jobs **2** machines](#Johnson-**n**-jobs-**2**-machines)




## Data utils

In [1]:
import numpy as np
import random
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from utils.benchmarks import benchmarks

### Path Cost calculation function :
Used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [2]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]

    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

### Gantt graph generator

In [3]:
def generate_gantt_chart(processing_times, seq, interval=50, labeled=True):
    data = processing_times.T
    nb_jobs, nb_machines = processing_times.shape
    schedules = np.zeros((nb_machines, nb_jobs), dtype=dict)
    # schedule first job alone first
    task = {"name": "job_{}".format(
        seq[0]+1), "start_time": 0, "end_time": data[0][seq[0]]}

    schedules[0][0] = task
    for m_id in range(1, nb_machines):
        start_t = schedules[m_id-1][0]["end_time"]
        end_t = start_t + data[m_id][0]
        task = {"name": "job_{}".format(
            seq[0]+1), "start_time": start_t, "end_time": end_t}
        schedules[m_id][0] = task

    for index, job_id in enumerate(seq[1::]):
        start_t = schedules[0][index]["end_time"]
        end_t = start_t + data[0][job_id]
        task = {"name": "job_{}".format(
            job_id+1), "start_time": start_t, "end_time": end_t}
        schedules[0][index+1] = task
        for m_id in range(1, nb_machines):
            start_t = max(schedules[m_id][index]["end_time"],
                          schedules[m_id-1][index+1]["end_time"])
            end_t = start_t + data[m_id][job_id]
            task = {"name": "job_{}".format(
                job_id+1), "start_time": start_t, "end_time": end_t}
            schedules[m_id][index+1] = task

    # create a new figure
    fig, ax = plt.subplots(figsize=(18, 8))

    # set y-axis ticks and labels
    y_ticks = list(range(len(schedules)))
    y_labels = [f'Machine {i+1}' for i in y_ticks]
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels)

    # calculate the total time
    total_time = max([job['end_time'] for proc in schedules for job in proc])

    # set x-axis limits and ticks
    ax.set_xlim(0, total_time)
    x_ticks = list(range(0, total_time+1, interval))
    ax.set_xticks(x_ticks)

    # set grid lines
    ax.grid(True, axis='x', linestyle='--')

    # create a color dictionary to map each job to a color
    color_dict = {}
    for proc in schedules:
        for job in proc:
            if job['name'] not in color_dict:
                color_dict[job['name']] = (np.random.uniform(
                    0, 1), np.random.uniform(0, 1), np.random.uniform(0, 1))

    # plot the bars for each job on each processor
    for i, proc in enumerate(schedules):
        for job in proc:
            start = job['start_time']
            end = job['end_time']
            duration = end - start
            color = color_dict[job['name']]
            ax.barh(i, duration, left=start, height=0.5,
                    align='center', color=color, alpha=0.8)
            if labeled:
                # add job labels
                label_x = start + duration/2
                label_y = i
                ax.text(
                    label_x, label_y, job['name'][4:], ha='center', va='center', fontsize=10)

    plt.show()

### ACO for FSP

In [58]:
class AntColonyOptimization:
    def __init__(self, processingTimes, Alpha, Beta, Q, max_it, num_ant, rho, heuristicSolution) -> None:
        self.numberJobs = processingTimes.shape[0]
        self.numberMachines = processingTimes.shape[1]
        self.Distances = np.zeros((self.numberJobs, self.numberJobs))
        self.processingTimes = processingTimes
        self.archive = heuristicSolution
        self.alpha = Alpha
        self.beta = Beta
        self.Q = Q
        self.globalPheromone = np.ones((self.numberJobs, self.numberJobs))
        self.maxIt = max_it
        self.numAnt = num_ant
        self.rho = rho

    def calculateDistances(self):
        for i in range(self.numberJobs):
            for j in range(self.numberJobs):
                if (i == j):
                    self.Distances[i, j] =0
                else : 
                    for k in range(self.numberMachines-1):
                         self.Distances[i, j] += self.processingTimes[j, k] + max(0, self.processingTimes[i, k+1]- self.processingTimes[j, k])
                    self.Distances[i, j] += self.processingTimes[j, self.numberMachines-1]  

    # calcule la formule de choix du job à prendre 
    def calculateJobVoisin(self, jobCourant, lePotentielJobVoisin, lesPotentielsJobsVoisins):
        denominateur = 0
        numerateur = 0
        for i in range(len(lesPotentielsJobsVoisins)):
            denominateur += (self.globalPheromone[jobCourant, lesPotentielsJobsVoisins[i]])**self.alpha * (1/self.Distances[jobCourant, lesPotentielsJobsVoisins[i]])**self.beta
        numerateur =  (self.globalPheromone[jobCourant, lePotentielJobVoisin])**self.alpha * (1/self.Distances[jobCourant, lePotentielJobVoisin])**self.beta
        # print("numerateur", numerateur)
        # print("denom", denominateur)
        return numerateur/denominateur

    def updateLocalPheromone(self, solutionSequence, localPheromoneMatrix):
        solutionQuality = calculate_makespan(self.processingTimes, solutionSequence)
        for j in range(len(solutionSequence)-1):
            localPheromoneMatrix[solutionSequence[j], solutionSequence[j+1]] += self.Q /solutionQuality
   
    def updateGlobalPheromone(self, localPheromoneMatrix):
        self.globalPheromone = (1-self.rho)*self.globalPheromone + localPheromoneMatrix          
    
    def run(self):
        self.calculateDistances()
        for it in range(self.maxIt):
            #liste de liste pour contenir les solutions de chaque fourmi
            solutions = []
            localPheromone = np.zeros((self.numberJobs, self.numberJobs))

            for ant in range(self.numAnt): 
                solutions.append([])
                num_job_pris = 0
                #initialisation de la liste contenant les jobs qui constitueront la solution, elle sera updaté à chaque fois qu'un job est pris
                job_dispo = list(range(self.numberJobs))
                # démarrer par un job aléatoirement
                solutions[ant].append(job_dispo[random.randint(0, len(job_dispo) - 1)])
                
                #updating ce qu'il faut
                job_dispo.remove(solutions[ant][num_job_pris])
                num_job_pris +=1
                while(len(job_dispo)>0): # équivaut à dire num_job_pris < self.numberJobs
                    bestVoisin = 0
                    nextJob = None
                    for i in range(len(job_dispo)):
                        #solutions[ant][num_job_pris-1] is the current node
                        voisinActuel = self.calculateJobVoisin(solutions[ant][num_job_pris-1], job_dispo[i], job_dispo)
                        if( voisinActuel > bestVoisin):
                            bestVoisin = voisinActuel
                            nextJob = job_dispo[i]

                    solutions[ant].append(nextJob)
                    #updating ce qu'il faut
                    job_dispo.remove(nextJob)
                    num_job_pris +=1
                #Mise à jour local de phéromone
                self.updateLocalPheromone(solutions[ant], localPheromone) 
                # print("Ant : ", ant, "'s solution : ", solutions[ant]) 
                # print("Its makespan is : ", calculate_makespan(self.processingTimes, solutions[ant]))
            #Mise à jour globale de phéromone 
            self.updateGlobalPheromone(localPheromone)
            # L'idée que j'ai est de calculer le makespan de toutes les solutions et si une est meilleure de celle contenu dans l'archive l'archiver à son tour 
            for sol in range(len(solutions)):
                makespan = calculate_makespan(self.processingTimes, solutions[sol])
                if (makespan < calculate_makespan(self.processingTimes, self.archive)):
                    self.archive = solutions[sol]
                

        #Retourner la meilleure solution 
        return self.archive  




In [6]:
matrix_data = [
    [2, 5, 3],
    [10, 6, 1],
    [4, 3, 2],
]

matrix_np = np.array(matrix_data)
ACO = AntColonyOptimization(matrix_np)
testDist = ACO.calculateDistances()
testDist

array([[ 0., 17., 10.],
       [14.,  0., 11.],
       [11., 17.,  0.]])

In [59]:
nehList = [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]
ACO = AntColonyOptimization(benchmarks[0], 0.5, 0.5, 0.9, 4, 16, 0, nehList )
ACO.run()

[[  0. 302. 239. 397. 355. 341. 355. 305. 288. 307. 332. 348. 284. 327.
  297. 306. 282. 343. 298. 323.]
 [325.   0. 238. 381. 386. 309. 321. 285. 299. 338. 394. 366. 230. 281.
  267. 367. 293. 346. 358. 325.]
 [288. 335.   0. 354. 353. 277. 285. 229. 251. 305. 327. 275. 197. 235.
  227. 300. 224. 343. 313. 270.]
 [389. 344. 287.   0. 382. 363. 404. 353. 336. 321. 353. 385. 333. 376.
  346. 354. 331. 363. 344. 357.]
 [347. 375. 296. 401.   0. 349. 339. 323. 345. 352. 414. 383. 284. 325.
  326. 384. 335. 349. 356. 342.]
 [364. 333. 261. 414. 355.   0. 348. 321. 324. 332. 383. 350. 281. 326.
  311. 340. 318. 359. 325. 325.]
 [318. 362. 281. 366. 379. 340.   0. 282. 338. 321. 343. 298. 296. 317.
  325. 346. 297. 355. 357. 297.]
 [322. 309. 203. 382. 353. 315. 313.   0. 260. 305. 329. 324. 245. 284.
  273. 287. 254. 343. 287. 299.]
 [309. 354. 210. 373. 354. 281. 334. 257.   0. 306. 353. 342. 227. 271.
  248. 339. 265. 343. 339. 317.]
 [344. 350. 256. 408. 361. 337. 306. 305. 323.   0. 396

[8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]